In [ ]:
# 1. Install dependencies
!pip install -q groq

import os
from groq import Groq

# 2. Setup API Key - REPLACE WITH YOUR ACTUAL KEY
GROQ_API_KEY = "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"
client = Groq(api_key=GROQ_API_KEY)

# 3. Define Expert Configurations
# Using Llama 3.3 70B for complex tasks and 8B for simple/fast routing
MODEL_CONFIG = {
    "technical": {
        "system_prompt": "You are a Rigorous Technical Expert. Focus on code precision and debugging. Use Markdown for code blocks.",
        "model": "llama-3.3-70b-versatile"
    },
    "billing": {
        "system_prompt": "You are an Empathetic Billing Expert. Focus on financial policies and refund windows (3-5 days).",
        "model": "llama-3.3-70b-versatile"
    },
    "general": {
        "system_prompt": "You are a friendly General Expert for casual conversation.",
        "model": "llama-3.1-8b-instant"
    },
    "tool_use": {
        "system_prompt": "You are a Tool Specialist. Provide specific data points or mock API results.",
        "model": "llama-3.1-8b-instant"
    }
}

# 4. The Router Function (Gating Network)
def route_prompt(user_input):
    router_instruction = (
        "Classify this query into one of these categories: [technical, billing, general, tool_use]. "
        "Return ONLY the word, nothing else."
    )

    try:
        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": router_instruction},
                {"role": "user", "content": user_input}
            ],
            model="llama-3.1-8b-instant", # Fast model for routing
            temperature=0 # Deterministic
        )
        return response.choices[0].message.content.strip().lower()
    except Exception as e:
        print(f"Routing Error: {e}")
        return "general"

# 5. Bonus: Mock Tool Function
def mock_tool_expert(user_input):
    if "bitcoin" in user_input.lower():
        return "The current mock price of Bitcoin is $98,450.25 (Simulated API Call)."
    return "Tool Expert: Data not found for this specific request."

# 6. The Orchestrator
def process_request(user_input):
    # Determine the expert
    category = route_prompt(user_input)
    print(f"--- [Router Decided: {category.upper()}] ---")

    # Handle the Bonus Challenge (Tool Use)
    if "tool_use" in category:
        return mock_tool_expert(user_input)

    # Select configuration
    config = MODEL_CONFIG.get(category, MODEL_CONFIG["general"])

    # Call the selected Expert
    try:
        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": config["system_prompt"]},
                {"role": "user", "content": user_input}
            ],
            model=config["model"],
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Expert Error: {e}"

# 7. Execution Loop
test_queries = [
    "My python script is throwing an IndexError on line 5.",
    "I was charged twice for my subscription this month.",
    "Hey! How's your day going?",
    "What is the current price of Bitcoin?"
]

print("Starting MoE Router System...\n")
for query in test_queries:
    print(f"User Input: {query}")
    result = process_request(query)
    print(f"Final Response:\n{result}")
    print("="*60 + "\n")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 5.4 MB/s eta 0:00:00
